In [54]:
!pip install watson_developer_cloud
from pandas.io.json import json_normalize
from watson_developer_cloud import  PersonalityInsightsV3
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk import word_tokenize, ngrams
from os import listdir
import pandas as pd
import statistics
import string
import random
import json


In [9]:
characters = ["RACHEL", "ROSS", "MONICA", "CHANDLER", "JOEY", "PHOEBE"]

personal_stopwords = ["yeah", "okay", "gon", "na", "gonna",
                      "oh", "hey", "well", "im", "right",
                      "like", "its"]
personal_stopwords = ["gon", "na"]

stop = stopwords.words('english') + personal_stopwords
no_punc = str.maketrans(string.punctuation, len(string.punctuation)*" ")
printable = set(string.printable)

def clean_line(text):
    return ''.join(list((filter(lambda x: x in printable, str(text)))))

In [3]:
term_frequencies = { character: {} for character in characters }
all_text = { character: [] for character in characters }
word_count = { character: 0 for character in characters }

print_count = 0

for transcript in listdir("./data/transcripts")[:]:
    df = pd.read_csv("./data/transcripts/{}".format(transcript))
    df.dropna()
    print(transcript.ljust(8), end=" ")
    
    if print_count == 10:
        print()
        print_count = 0
    else:
        print_count += 1
        
    for character in characters:
        for line in df[df["Speaker"].str.contains(character)]["Line"]:
            char_term_frequencies = term_frequencies[character]
            line = clean_line(line)
            all_text[character].append(line)
            tokens = word_tokenize(line.translate(no_punc))
            for word in tokens:
                word_count[character] += 1
                if word in char_term_frequencies:
                    char_term_frequencies[word] += 1
                else:
                    char_term_frequencies[word] = 1

1001.csv 1002.csv 1003.csv 1004.csv 1005.csv 1006.csv 1007.csv 1008.csv 1009.csv 101.csv  1010.csv 
1011.csv 1012.csv 1013.csv 1014.csv 1015.csv 1016.csv 1017.csv 102.csv  103.csv  104.csv  105.csv  
106.csv  107.csv  108.csv  109.csv  110.csv  111.csv  112.csv  113.csv  114.csv  115.csv  116.csv  
117.csv  118.csv  119.csv  120.csv  121.csv  122.csv  123.csv  124.csv  201.csv  202.csv  203.csv  
204.csv  205.csv  206.csv  207.csv  208.csv  209.csv  210.csv  211.csv  212.csv  214.csv  215.csv  
216.csv  217.csv  218.csv  219.csv  220.csv  221.csv  222.csv  223.csv  224.csv  301.csv  302.csv  
303.csv  304.csv  305.csv  306.csv  307.csv  308.csv  309.csv  310.csv  311.csv  312.csv  313.csv  
314.csv  315.csv  316.csv  317.csv  318.csv  319.csv  320.csv  321.csv  322.csv  323.csv  324.csv  
325.csv  401.csv  402.csv  403.csv  404.csv  405.csv  406.csv  407.csv  408.csv  409.csv  410.csv  
411.csv  412.csv  413.csv  414.csv  415.csv  416.csv  417.csv  418.csv  419.csv  420.csv  421.csv  


In [10]:
common_words = []
with open('./data/word_count.json', 'w') as count_json:
    count_json.write('[\n')
    
    for character in characters:
        char_term_freqs = term_frequencies[character]
        percent_words = round((word_count[character]/sum(word_count.values()))*100, 2)
        print("{} said {} words ({}%)".format(character,
                                              word_count[character],
                                              percent_words))
        print("{}'s top 10 words were".format(character))
        
        sorted_words = sorted(char_term_freqs, key=char_term_freqs.get, reverse=True)
        ith_word = 0
        to_print = True
        char_output = {
            "name": character,
            "percent": percent_words,
            "uniqueWords": len(char_term_freqs.keys()),
            "words": {}
        }
            
        for word in sorted_words:
            if word not in stop:
                if (to_print):
                    print("{} was said {} times".format(word, char_term_freqs[word]))
                common_words.append(word)
                char_output["words"][word] = char_term_freqs[word]
                ith_word += 1
            if ith_word == 10:
                to_print = False
            if ith_word == 250:
                break
                
        json.dump(char_output, count_json, indent=4)
        count_json.write(',\n')
        print()
    common = set([x for x in common_words if common_words.count(x) > 1])
    common_occurences = {}
    for word in common:
        occurences = 0
        for character in characters:
            occurences += term_frequencies[character][word]
        common_occurences[word] = occurences
    common_output = {
        "name": "COMMON",
        "percent": 100,
        "uniqueWords": 100,
        "words": common_occurences
    }
    print(len(common_output["words"]))
    json.dump(common_output, count_json, indent=4)
    count_json.write(']')

RACHEL said 102239 words (18.18%)
RACHEL's top 10 words were
oh was said 1936 times
know was said 1051 times
okay was said 852 times
well was said 825 times
yeah was said 804 times
right was said 556 times
im was said 553 times
ross was said 523 times
god was said 484 times
really was said 455 times

ROSS said 100824 words (17.92%)
ROSS's top 10 words were
oh was said 980 times
know was said 886 times
yeah was said 856 times
okay was said 844 times
hey was said 778 times
uh was said 747 times
well was said 603 times
right was said 514 times
im was said 506 times
like was said 446 times

MONICA said 88174 words (15.68%)
MONICA's top 10 words were
oh was said 978 times
okay was said 675 times
know was said 663 times
well was said 453 times
right was said 441 times
hey was said 431 times
yeah was said 417 times
get was said 406 times
go was said 406 times
really was said 394 times

CHANDLER said 91935 words (16.34%)
CHANDLER's top 10 words were
oh was said 811 times
know was said 802 time

In [5]:
freqs = [value for value in word_count.values()]
median = statistics.median(freqs)
with open('./data/most-talkative.json', 'w') as most_talkative:
    most_talkative.write("[\n")
    median_json = {
        "name": "median",
        "words": median
    }
    json.dump(median_json, most_talkative, indent=4)
    most_talkative.write(",\n")
    for character in characters:
        difference = round((word_count[character]/median) - 1, 3)
        print("{} speaks {}% less/more than than the median".format(character, difference))
        char_output = {
            "name": character,
            "words": word_count[character],
            "frequency": difference
        }
        json.dump(char_output, most_talkative, indent=4)
        if character is not characters[-1]:
            most_talkative.write(",\n")
    most_talkative.write("]")
        
    
print()

for character in characters:
    print("{} says {} lines".format(character, len(all_text[character])))

RACHEL speaks 0.107% less/more than than the median
ROSS speaks 0.092% less/more than than the median
MONICA speaks -0.045% less/more than than the median
CHANDLER speaks -0.004% less/more than than the median
JOEY speaks 0.004% less/more than than the median
PHOEBE speaks -0.063% less/more than than the median

RACHEL says 9434 lines
ROSS says 9246 lines
MONICA says 8618 lines
CHANDLER says 8621 lines
JOEY says 8452 lines
PHOEBE says 7667 lines


In [5]:
network = {}

for character in characters:
    network[character] = {
        "Total": 0
    }
for source in characters:
    for dest in characters:
        if source is dest:
            continue
        dest = dest.lower()
        mentions = term_frequencies[source][dest]
        if dest is "phoebe":
            mentions += term_frequencies[source]["pheebs"]
        if dest is "rachel":
            mentions += term_frequencies[source]["rach"]
        if dest is "monica":
            #"c'mon is cheaper to parse here than it is in building the term_frequencies
            mentions += term_frequencies[source]["mon"] - term_frequencies[source]["c"] 
        if dest is "joey":
            mentions += term_frequencies[source]["joe"]
        network[source][dest.upper()] = mentions
        network[source]["Total"] += mentions
    
    for dest in characters:
        if source is dest:
            continue
        mentions = network[source][dest]
        percent_mentions = round((mentions/network[source]["Total"])*100)
        print("{} -> {} {} times ({}%)".format(source.title().ljust(8),
                                               dest.title().ljust(8),
                                               str(mentions).ljust(3),
                                               percent_mentions))
    print()

Rachel   -> Ross     523 times (37%)
Rachel   -> Monica   215 times (15%)
Rachel   -> Chandler 149 times (10%)
Rachel   -> Joey     333 times (23%)
Rachel   -> Phoebe   205 times (14%)

Ross     -> Rachel   235 times (30%)
Ross     -> Monica   135 times (17%)
Ross     -> Chandler 135 times (17%)
Ross     -> Joey     175 times (22%)
Ross     -> Phoebe   99  times (13%)

Monica   -> Rachel   188 times (16%)
Monica   -> Ross     205 times (18%)
Monica   -> Chandler 283 times (24%)
Monica   -> Joey     229 times (20%)
Monica   -> Phoebe   259 times (22%)

Chandler -> Rachel   66  times (9%)
Chandler -> Ross     178 times (24%)
Chandler -> Monica   197 times (27%)
Chandler -> Joey     220 times (30%)
Chandler -> Phoebe   70  times (10%)

Joey     -> Rachel   118 times (14%)
Joey     -> Ross     337 times (39%)
Joey     -> Monica   122 times (14%)
Joey     -> Chandler 213 times (25%)
Joey     -> Phoebe   78  times (9%)

Phoebe   -> Rachel   149 times (19%)
Phoebe   -> Ross     184 times (23%

In [7]:
nodes = list(map(lambda name: {"name": name}, characters))
links = []

for source in characters:
    for dest in characters:
        if source is dest:
            continue
        mentions = network[source][dest]
        links.append({
            "source": source,
            "target": dest,
            "value": mentions,
            "percent": round((mentions/network[source]["Total"])*100)
        })
graph = [nodes, links]
with open('./data/network.json', 'w') as outfile:
    json.dump(graph, outfile, indent=4)

In [8]:
personality_insights = PersonalityInsightsV3(
    username=PERSONALITY_USERNAME,
    password=PERSONALITY_PASSWORD,
    version='2016-10-20')

for character in characters:
    char_all_text = all_text[character]
    random.shuffle(char_all_text)
    joined_text = ' '.join(char_all_text)
    json_response = personality_insights.profile(joined_text)
    with open('./data/personality-{}.json'.format(character), 'w') as outfile:
        json.dump(json_response, outfile, indent=4)

In [84]:
personalities = {character: {} for character in characters}

for character in characters:
    with open('./data/personality-{}.json'.format(character), 'r') as data:
        personality = json.load(data)
        for trait in personality["personality"]:
            personalities[character][trait["name"]] = trait["percentile"]
            for child in trait["children"]:
                personalities[character][child["name"]] = child["percentile"]
            
traits = personalities[list(personalities.keys())[0]].keys()

for trait in traits:
    print(trait)
    for character in characters:
        print("{}: {}".format(character.title(), round(personalities[character][trait],2)))
    print()

Fiery
Rachel: 0.11
Ross: 0.09
Monica: 0.1
Chandler: 0.1
Joey: 0.07
Phoebe: 0.11

Authority-challenging
Rachel: 0.49
Ross: 0.54
Monica: 0.45
Chandler: 0.55
Joey: 0.52
Phoebe: 0.47

Conscientiousness
Rachel: 0.81
Ross: 0.79
Monica: 0.86
Chandler: 0.79
Joey: 0.82
Phoebe: 0.81

Activity level
Rachel: 0.08
Ross: 0.09
Monica: 0.11
Chandler: 0.11
Joey: 0.1
Phoebe: 0.07

Uncompromising
Rachel: 1.0
Ross: 1.0
Monica: 1.0
Chandler: 0.99
Joey: 0.99
Phoebe: 0.99

Orderliness
Rachel: 0.32
Ross: 0.37
Monica: 0.4
Chandler: 0.42
Joey: 0.54
Phoebe: 0.45

Gregariousness
Rachel: 0.28
Ross: 0.27
Monica: 0.4
Chandler: 0.33
Joey: 0.43
Phoebe: 0.27

Imagination
Rachel: 0.02
Ross: 0.03
Monica: 0.03
Chandler: 0.04
Joey: 0.04
Phoebe: 0.04

Immoderation
Rachel: 0.36
Ross: 0.33
Monica: 0.42
Chandler: 0.39
Joey: 0.41
Phoebe: 0.32

Intellect
Rachel: 0.53
Ross: 0.72
Monica: 0.64
Chandler: 0.69
Joey: 0.54
Phoebe: 0.57

Modesty
Rachel: 1.0
Ross: 1.0
Monica: 1.0
Chandler: 0.99
Joey: 0.99
Phoebe: 1.0

Dutifulness
Rachel:

In [85]:
sentiments = {}
sid = SentimentIntensityAnalyzer()
sentiment_sum = 0
sentiment_count = 0
for character in characters:
    char_all_text = all_text[character]
    sentiments[character] = {
        "positive": 0,
        "neutral": 0,
        "negative": 0,
        "total": len(char_all_text)
    }
    print("{} ".format(character.title()), end="")
    for line in char_all_text:
        sentiment = sid.polarity_scores(line)["compound"]
        sentiment_sum += sentiment
        sentiment_count += 1
        if sentiment < -0.5:
            key = "negative"
        elif sentiment < 0.5:
            key = "neutral"
        else:
            key = "positive"
        sentiments[character][key] += 1

Rachel Ross Monica Chandler Joey Phoebe 

In [96]:
print("Average: {}\n".format(round(sentiment_sum/sentiment_count, 2)))

for character in characters:
    sent = sentiments[character]
    positive = round((sent["positive"]/sent["total"])*100, 2)
    neutral  = round((sent["neutral"]/sent["total"]) *100, 2)
    negative = round((sent["negative"]/sent["total"])*100, 2)
    print(character.title())
    print("Positive: {} ({}%)".format(sent["positive"], positive))
    print("Neutral:  {} ({}%)".format(sent["neutral"], neutral))
    print("Negative: {}  ({}%)\n".format(sent["negative"], negative))
    
with open('./data/sentiments.json', 'w') as outfile:
        json.dump(sentiments, outfile, indent=4)

Average: 0.13

Rachel
Positive: 1741 (18.45%)
Neutral:  7198 (76.3%)
Negative: 495  (5.25%)

Ross
Positive: 1559 (16.86%)
Neutral:  7205 (77.93%)
Negative: 482  (5.21%)

Monica
Positive: 1347 (15.63%)
Neutral:  6804 (78.95%)
Negative: 467  (5.42%)

Chandler
Positive: 1381 (16.02%)
Neutral:  6757 (78.38%)
Negative: 483  (5.6%)

Joey
Positive: 1443 (17.07%)
Neutral:  6534 (77.31%)
Negative: 475  (5.62%)

Phoebe
Positive: 1501 (19.58%)
Neutral:  5688 (74.19%)
Negative: 478  (6.23%)



In [12]:
# cross-filterable word clouds
# force-directed bezier graphs of the social network
# bubble chart of who talks the most, normalized
# sentiment grouped bar chart w/ error analysis
# tri-gram map
# personality radar charts with big 5 breakdown

In [16]:
def fix_unicode_gaps(text):
    text = text.replace("don t", "dont")
    text = text.replace("you re", "youre")
    text = text.replace("i m", "im")
    text = text.replace("i ll", "ill")
    text = text.replace("we re", "were")
    text = text.replace("it s", "its")
    text = text.replace("can t", "cant")
    text = text.replace("won t", "wont")
    text = text.replace("that s", "thats")
    text = text.replace("didn t", "didnt")
    text = text.replace("y know", "yknow")
    return text

def repair_nltk_contractions(text):
    text = text.replace("gon na", "gonna")
    text = text.replace("wan na", "wanna")
    text = text.replace("got ta", "gotta")
    text = text.replace("they ll", "theyll")
    text = text.replace("smell ly", "smelly")
    return text

gram_size = 3
output_dict = {}
for character in characters:
    ngram_list = []
    for line in all_text[character]:
        tokens = word_tokenize(fix_unicode_gaps(line.translate(no_punc)))
        ngram_list.extend(list(ngrams(tokens, gram_size)))
    ngram_dict = {}
    for gram in ngram_list:
        phrase = ' '.join(gram)
        if phrase in ngram_dict:
            ngram_dict[phrase] += 1
        else:
            ngram_dict[phrase] = 1
    sorted_phrases = sorted(ngram_dict.items(), key=lambda x: x[1], reverse=True)
    print(character.title())
    for phrase in sorted_phrases[:5]:
        joined_phrase = repair_nltk_contractions(phrase[0])
        print("\"{}\" was said {} times".format(joined_phrase, phrase[1]))
    print()

Rachel
"oh my god" was said 256 times
"i dont know" was said 138 times
"no no no" was said 135 times
"im gonna" was said 120 times
"you know what" was said 100 times

Ross
"i dont know" was said 122 times
"oh my god" was said 114 times
"im gonna" was said 102 times
"no no no" was said 102 times
"gonna be" was said 88 times

Monica
"oh my god" was said 232 times
"what are you" was said 98 times
"im gonna" was said 89 times
"gonna be" was said 88 times
"i cant believe" was said 72 times

Chandler
"no no no" was said 199 times
"im gonna" was said 92 times
"i dont know" was said 85 times
"gonna be" was said 82 times
"what are you" was said 72 times

Joey
"no no no" was said 188 times
"i dont know" was said 136 times
"im gonna" was said 101 times
"gonna be" was said 82 times
"what are you" was said 67 times

Phoebe
"oh my god" was said 173 times
"im gonna" was said 92 times
"i dont know" was said 87 times
"no no no" was said 68 times
"gonna be" was said 58 times

